In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import librosa
import IPython.display as ipd
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import glob
import warnings
import librosa
import os
from sklearn.model_selection import train_test_split
import numpy as np
# from tqdm import tqdm
from keras import optimizers
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'keras'

In [15]:
import sys
print(sys.version)

3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)]


In [3]:
keras.__version__

'2.8.0'

In [4]:
print(tf.__version__)

2.8.0


In [5]:
import librosa
import os
from sklearn.model_selection import train_test_split
import numpy as np
# from tqdm import tqdm
from keras import optimizers

In [6]:
MFCC_NUM = 40
MFCC_MAX_LEN = 2000
SAMPLING_RATE = 44100
feature_dim_1 = MFCC_NUM
feature_dim_2 = MFCC_MAX_LEN
channel = 1

In [7]:
emotions_ravdess={
  '01': 1,
  '02': 2,
  '03': 3,
  '04': 4,
  '05': 5,
  '06': 6,
  '07': 7,
  '08': 8
}


emotions_cremad={
  'ANG':5,
  'DIS':7,
  'NEU':1,
  'HAP':3,
  'FEA':6,
  'SAD':4
}

emotions_to_string={
  1:'neutral',
  2:'calm',
  3:'happy',
  4:'sad',
  5:'angry',
  6:'fearful',
  7:'disgust',
  8:'surprised'
}

emotions_intensity = {
  '01':'low',
  '02':'strong',
}

observed_emotions=[1 , 5,  3]

In [8]:
def extract_feature(wave, max_len=MFCC_MAX_LEN):
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [9]:
X = []
y = []

def append_X_Y(label, wave):
    y.append(label)
    mfcc = extract_feature(wave)
    X.append(mfcc)

In [10]:
for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\ravdess\\Actor_*\\*.wav"):
    file_name=os.path.basename(file)
    label=emotions_ravdess[file_name.split("-")[2]] 
    if emotions_ravdess[file_name.split("-")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, mono=True, sr=44100)
    wave = wave[::3]
    append_X_Y(label, wave)
    

In [11]:
X = np.array(X)
y = np.array(y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=True, shuffle=True)

In [13]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(120, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [14]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_split=0.25)

Epoch 1/10
6/6 [==============================] - 5s 855ms/step - loss: 8.6940 - accuracy: 0.3665 - val_loss: 1.3048 - val_accuracy: 0.4844
Epoch 2/10
6/6 [==============================] - 5s 831ms/step - loss: 1.0798 - accuracy: 0.5236 - val_loss: 1.0197 - val_accuracy: 0.3750
Epoch 3/10
6/6 [==============================] - 5s 822ms/step - loss: 0.9705 - accuracy: 0.4555 - val_loss: 0.9184 - val_accuracy: 0.5312
Epoch 4/10
6/6 [==============================] - 5s 837ms/step - loss: 0.8335 - accuracy: 0.5916 - val_loss: 0.8859 - val_accuracy: 0.5625
Epoch 5/10
6/6 [==============================] - 11s 2s/step - loss: 0.8828 - accuracy: 0.5864 - val_loss: 0.8060 - val_accuracy: 0.6094
Epoch 6/10
6/6 [==============================] - 5s 854ms/step - loss: 0.6904 - accuracy: 0.7225 - val_loss: 0.8773 - val_accuracy: 0.5625
Epoch 7/10
6/6 [==============================] - 5s 825ms/step - loss: 0.7559 - accuracy: 0.5812 - val_loss: 1.0368 - val_accuracy: 0.4844
Epoch 8/10
6/6 [======